In [1]:
#-------------------------------------------------------------------------------
#@markdown > #### **Yeti**common
#@markdown Run this cell on the first run and after a restart

#-------------------------------------------------------------------------------
#MOUNT // Google Drive
import os, shutil
from pathlib import Path
from google.colab import drive
import sys, time, warnings, glob, itertools
from IPython.display import clear_output, display, Image
from datetime import datetime
from random import randrange
from slugify import slugify
############################################################
#DEBUG
GIT_RESYNC=0
if GIT_RESYNC:
  !rm -rf -rf /content/yeti /content/yeti.py
  !git clone https://github.com/keirVQGAN/yeti /content/yeti
  shutil.copy('/content/yeti/yeti.py','/content') 
  clear_output()


#-------------------------------------------------------------------------------
#MOUNT // Google Drive
if not os.path.ismount('/content/drive'): drive.mount ('/content/drive/')
if os.path.isdir('/content/sample_data'): shutil.rmtree('/content/sample_data')
clear_output()


#-------------------------------------------------------------------------------
#VARIABLES // Global
localPath='/content'
drivePath='/content/drive/MyDrive/yeti'
inPath=f'{drivePath}/in'
outPath=f'{drivePath}/out'
start_time = time.time()
warnings.filterwarnings('ignore')
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
confPath='/content/config/conf'


#-------------------------------------------------------------------------------
#INSTALL // yetiCommon
if not os.path.isdir('/content/yeti'):
  !git clone https://github.com/keirVQGAN/yeti /content/yeti                    &> /dev/null
  shutil.copy('/content/yeti/yeti.py','/content')
  %pip install -r /content/yeti/requirements_yeti.txt                           &> /dev/null
  import yeti
  yetiPaths=['super','trans','inter','video','csv', 'style']
  for y in yetiPaths: yeti.mk(f'{inPath}/{y}')
  
else: import yeti
yeti.txtY('Installed','yetiUncommon')
yeti.txtC('GPU', gpu[1])
yeti.timeTaken(start_time)

>> Installed | yetiUncommon

>> GPU | Tesla T4

>> Complete: | 7 Seconds

#####YetiDiffuse

In [ ]:
#-------------------------------------------------------------------------------
#@markdown > #### **Yeti**Diffuse
#-------------------------------------------------------------------------------
#INSTALL stable diffusion
if os.path.isdir('/temp/hugged'):
  yeti.txtC('Huggingface API','🤗')
else:
  yeti.txtM('Installing', 'yetiDiffuse')
  !bash /content/yeti/ini/yetiDiffuse_ini.sh &> /dev/null
  from google.colab import output
  output.enable_custom_widget_manager()
  from huggingface_hub import notebook_login
  #🤗 = > Uncomment below for new token
  #notebook_login()
  !mkdir /root/.huggingface &> /dev/n
  !cp /content/yeti/hugToken /root/.huggingface/token
  yeti.mk('/temp/hugged')
  yeti.txtC('Huggingface API','🤗')

#-------------------------------------------------------------------------------
#Install Diffusers
if os.path.isdir('/root/.cache/huggingface/diffusers/'):
  yeti.txtY('Pretrained Model', 'Downloaded')
else:
  import torch
  from diffusers import StableDiffusionPipeline
  yeti.txtY('Pretrained Model', 'Downloading...')
  pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
  clear_output()
  pipe = pipe.to("cuda")

if not os.path.isdir('/content/sdutils'):
  yeti.txtM('Installing', 'sdutils...')
  !git clone https://github.com/newsbubbles/sdutils &> /dev/null
  !cp /content/sdutils/*.py /content/ &> /dev/null
yeti.txtM('Installed', 'sdutils')

#Remove NSFW - Function
def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

#-------------------------------------------------------------------------------
#Install prompt generator
from promptgen import PromptGenerator
from genutil import Scaffold
scaffold = Scaffold("cuda", "PNDMScheduler", pipe)

#-------------------------------------------------------------------------------
#Settings // Defaults
PROMPTER = "An award winning $shot studio photograph of an art student $drape in a cone of the future $object made of $material, shot with a $camera, lit by $lighting" #@param {type:"string"}
JSON_PROMPT = 'cone_r1' #@param {type:"string"}
jsonFile = f'/content/yeti/prompts/{JSON_PROMPT}.json'
data, text_prompt, strength, prompt_data = yeti.prompter(jsonFile,PROMPTER)
PROMPT=text_prompt
now = datetime.now()
timeNow = now.strftime("%a-%H")
WIDTH = 512
HEIGHT = 512

#-------------------------------------------------------------------------------
#Settings // User
STEPS =100 #@param {type:"number"}
VARIATIONS = 100 #@param {type:"number"}
GUIDANCE = 8 #@param {type: "slider", min: 0.5, max: 9.5, step:0.5}
RANDOM_SEED = True #@param {type:"boolean"}
if RANDOM_SEED:
  SEED_NUMBER=randrange(0, 10000, 1)
else:
  SEED_NUMBER = 417 #@param{type: "slider", min: 1, max: 1000}
SEED_MAN = torch.Generator("cuda").manual_seed(SEED_NUMBER)
SCHEDULER = 'PNDMScheduler' #@param ["LMSDiscreteScheduler", "PNDMScheduler", "DDIMScheduler"]

#-------------------------------------------------------------------------------
#RENDER (Prompt Batch)

for i in range(VARIATIONS):
  data, text_prompt, strength, prompt_data = yeti.prompter(jsonFile,PROMPTER)
  PROMPT=text_prompt
  PROMPT_SLUGED=slugify(PROMPT)
  PROMPT_SLUG=PROMPT_SLUGED[:125]
  SEED_NUMBER=randrange(0, 10000, 1)
  SEED_MAN = torch.Generator("cuda").manual_seed(SEED_NUMBER)
  localOut=f'{outPath}/diffuse/{timeNow}/{PROMPT_SLUG}'
  localOutFile=f"{localOut}/st{STEPS}_g{GUIDANCE}_s{SEED_NUMBER}_{SCHEDULER}.png"
  yeti.txtC(f'Rendering >> {PROMPT}',f'Scale:{HEIGHT}x{WIDTH} Steps:{STEPS}, Guidance:{GUIDANCE},Seed:{SEED_NUMBER}')
  #RENDER PIPE
  image = pipe(PROMPT, height=WIDTH, width=HEIGHT, num_inference_steps = STEPS, guidance_scale = GUIDANCE, scheduler = SCHEDULER, generator = SEED_MAN).images[0]
  #SAVE OUTPUT IMAGE AND PROMPT TO DRIVE
  yeti.mk(localOut)
  image.save(localOutFile)
  with open(f"{localOut}/prompt.txt", "w") as text_file:
    print(f'{PROMPT},Scale:{HEIGHT}x{WIDTH} Steps:{STEPS}, Guidance:{GUIDANCE},Seed:{SEED_NUMBER}', file=text_file)
  yeti.txtY(text_prompt,localOutFile)
  display(image)

>> Rendering >> An award winning full-body shot studio photograph of an art student enclosed in a cone of the 
future street-wear made of flowers, shot with a Panasonic Lumix DMC-G1, lit by high-key lighting | Scale:512x512 
Steps:100, Guidance:8,Seed:9032

  0%|          | 0/101 [00:00<?, ?it/s]

In [11]:
def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

### Scripts

##### **Yeti**Txt2img - Pytti5

In [ ]:
#-------------------------------------------------------------------------------
#@markdown > #### **Yeti**txt2img // Render
#-------------------------------------------------------------------------------
#DEBUG
RENDER=True
#-------------------------------------------------------------------------------
#INSTALL // Pytti-Core
start_time = time.time()
if not os.path.isdir('/content/pytti-core'):
  yeti.txtM('Installing','yetiTxt2img')
  !bash /content/yeti/ini/txt2img_ini.sh &> /dev/null 

!rm -rf /content/masks /content/images_out /content/config/conf
import pandas as pd
from logging import CRITICAL
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from loguru import logger
from omegaconf import OmegaConf
from pytti.workhorse import _main as render_frames

logger.remove()
logger.add(sys.stderr, level=CRITICAL)
pytti_panna_output = {}
clear_output()
%matplotlib inline
yeti.txtY('Installed','yetiTxt2img')
yeti.timeTaken(start_time)


#-------------------------------------------------------------------------------
#VARIABLE // User Input
INIT_IMAGE= 'lizz.tiff' #@param{type: "string"}
CSV = 'e' #@param{type: "string"}
QUALITY = "test" #@param ["test", "draft", "proof"]
THRESHOLD_MASK=True #@param {type:"boolean"}
THRESHOLD_LEVEL=130 #@param {type: "slider", min: 20, max: 220, step: 10}

init_image=f'{inPath}/init/{INIT_IMAGE}'
imageName = os.path.splitext(INIT_IMAGE)[0]
csv_file=f'{inPath}/csv/{CSV}.csv' 
thresholdFile=f'/content/masks/{imageName}/{imageName}-{THRESHOLD_LEVEL}_mask.jpg'

CONFIG_BASE_PATH='./config'
CONFIG='yetiDefault.yaml'
confPath='/content/config/conf'
CONFIG_PATH=f'/content/config/{CONFIG}'

if gpu=='Tesla T4': shutil.copy ('/content/yeti/yetiDefaultT4.yaml',CONFIG_PATH)
else: shutil.copy ('/content/yeti/yetiDefault.yaml',CONFIG_PATH)


#-------------------------------------------------------------------------------
#WRITE // YML
yeti.yml(QUALITY, csv_file, confPath, init_image)
CONFS=yeti.lsName(confPath)


#-------------------------------------------------------------------------------
#WRITE // THRESHOLD MASKS
threshMasked = yeti.thresh(init_image,localPath,INIT_IMAGE)


#-------------------------------------------------------------------------------
#RENDER // Pytti-Core
if RENDER:
  start_time = time.time()
  yeti.txtC('YMLs',CONFS)
  timeNow=time.strftime("%H-%M") 
  for CONF in CONFS:
    with initialize(config_path=CONFIG_BASE_PATH,version_base=None):
        if THRESHOLD_MASK: cfg = compose(config_name=CONFIG,overrides=[f"conf={CONF}",f"scene_suffix=:3_\[{thresholdFile}\]"])
        else: cfg = compose(config_name=CONFIG,overrides=[f"conf={CONF}"])
        yeti.txtC('Rendering',CONF)
        render_frames(cfg)
    yeti.outSync(localPath,outPath,timeNow)
    clear_output()
  yeti.txtC('Rendered',CONFS)
  yeti.timeTaken(start_time)


#####**Yeti**Super

In [ ]:
#-------------------------------------------------------------------------------
#@markdown > #### **yeti**Super
#-------------------------------------------------------------------------------
#PARAMETERS // User Input
subFolder='truss'#@param {type: "string"}
superIn=f'{inPath}/super/{subFolder}'
superOut=f'{outPath}/super/{subFolder}'
superTile=8 #@param {type: 'slider'}
RENDER=True#@param {type: "boolean"}

#-------------------------------------------------------------------------------
#INSTALL // yetiSuper
if not os.path.isdir('/content/Real-ESRGAN'):
  start_time = time.time()
  yeti.txtY('Installing','yetiSuper')
  !bash /content/yeti/ini/yetiSuper.sh                                          &> /dev/null
  yeti.timeTaken(start_time)

#-------------------------------------------------------------------------------
#RENDER // yetiSuper
if RENDER:
  yeti.mk(superOut)
  %cd /content/Real-ESRGAN/
  !python inference_realesrgan.py -n RealESRGAN_x4plus -i $superIn -o $superOut --outscale 4 --fp32 --suffix '_super' -t $superTile
  yeti.timeTaken(start_time)

#####**yeti**Interpolate

In [ ]:
#@markdown > #### **yeti**Interpolate
#-------------------------------------------------------------------------------
# INSTALL / yetiInterpolate
if not os.path.isdir('/content/frame_interpolation'):
  start_time = time.time()
  yeti.txtC('Installing', 'yetiInterpolate')
  !bash /content/yeti/ini/yetiInterpolate.sh                                    >/dev/null 2>&1
  !bash /yeti/ini/ffmpegCUDA.sh                                                 >/dev/null 2>&1
  yeti.timeTaken(start_time)

#-------------------------------------------------------------------------------
#PARAMETERS
timeNow = datetime.now().strftime("%H-%M-%S")
interPath='head'#@param{type:"string"}
interIn=f'{drivePath}/in/inter/{interPath}'
interRm=f'{drivePath}/in/inter/*'
interOut=f'{drivePath}/out/inter/{timeNow}/'
times =  8#@param #1=2x , 2=4x , 3=8x etc.
fps =  30#@param
path = '/content/frame_interpolation/pretrained_models/film_net/Style/saved_model'
RENDER=True #@param{type:"boolean"}
SPLIT=False #@param{type:"boolean"}

yeti.txtM('Running on:', gpu[1])

if SPLIT:
  splitIn=f'{inPath}/split/'
  splitFiles=glob.glob(f'{splitIn}/*')
  for splitFile in splitFiles:
    splitName=Path(splitFile).stem
    splitPath=f'{outPath}/split/{splitName}-frames'
    splitOut=f'{splitPath}/frame%05d.png'
    yeti.mk(splitPath)
    yeti.txtY('Splitting', splitFile)
    start_time = time.time()
    !ffmpeg -i $splitFile $splitOut -hide_banner -loglevel error
    yeti.txtC('Split', splitFile)
    yeti.timeTaken(start_time)

if RENDER:
  start_time = time.time()
  !python3 -m frame_interpolation.eval.interpolator_cli \
      --pattern $interIn \
      --fps $fps \
      --model_path $path \
      --times_to_interpolate $times \
      --output_video
  yeti.timeTaken(start_time)

  timeNow = datetime.now().strftime("%H-%M-%S")
  interOut=f'{drivePath}/out/inter/{interPath}'
  !mkdir -p $interOut
  shutil.move(f'{interIn}/interpolated_frames', f'{interOut}/interpolated_frames/{timeNow}-interpolated_frames')
  shutil.move(f'{interIn}/interpolated.mp4', f'{interOut}/{timeNow}-interpolated_x{times}-fps{fps}.mp4')

#####**Yeti**Ken

In [ ]:
#-------------------------------------------------------------------------------
#@markdown > ####  **yeti**Ken
#-------------------------------------------------------------------------------
#INSTALL // yetiKen
try:
  import chainer
except ImportError as error:
  start_time = time.time()
  yeti.txtC('Installing', 'yetiKen')
  %cd /content
  !bash /content/yeti/ini/yetiKen.sh                                            >/dev/null 2>&1
  import chainer
  import gevent
  import moviepy
  yeti.timeTaken(start_time)

#-------------------------------------------------------------------------------
#PARAMETERS // User Input
WIDTH=2160#@param
SHIFT=300#@param
ZOOM=1.2#@param
ZOOMmain=1#@param
SECONDS=15#@param
FPS=30#@param
LENGTH=SECONDS*FPS
RENDER=True#@param {type: "boolean"}

#-------------------------------------------------------------------------------
#PARAMETERS // Batch Render
BATCH=True#@param {type: "boolean"}
ZOOMS=[1.1,1.2,1.3]
SHIFTS=[100,300,500]
ZOOMmains=[1]

#-------------------------------------------------------------------------------
#PATHS
if not BATCH:
    if RENDER:
      os.chdir('/content/3d-ken-burns')
      yeti.kenConf(WIDTH, SHIFT, ZOOM, FPS, LENGTH, ZOOMmain)
      kenPaths=yeti.ken(inPath, outPath, WIDTH, FPS, SHIFT,ZOOM, SECONDS, ZOOMmain)
      kenOut=kenPaths[0]
      kenIn=kenPaths[1]
      for IN,OUT in zip(kenIn,kenOut):
        yeti.txtY('Rendering', f'{Path(OUT).stem}.mp4')
        !python autozoom.py --in $IN --out $OUT
#-------------------------------------------------------------------------------
#Render
if BATCH:
  if RENDER:
    os.chdir('/content/3d-ken-burns')
    for ZOOMmain in ZOOMmains:
      for ZOOM in ZOOMS:
        for SHIFT in SHIFTS:
          yeti.kenConf(WIDTH, SHIFT, ZOOM, FPS, LENGTH, ZOOMmain)
          kenPaths=yeti.ken(inPath, outPath, WIDTH, FPS, SHIFT,ZOOM, SECONDS, ZOOMmain)
          kenOut=kenPaths[0]
          kenIn=kenPaths[1]
          for IN,OUT in zip(kenIn,kenOut):
            yeti.txtY('Rendering:', f'{Path(OUT).stem}.mp4')
            yeti.txtC('Zoom',ZOOM)
            yeti.txtM('SHIFT',SHIFT)
            yeti.txtY('ZoomMain',ZOOMmain)
            !python autozoom.py --in $IN --out $OUT

#####**Yeti**Transfer

In [ ]:
#@markdown > ##### **yeti**Transfer
#-------------------------------------------------------------------------------
force_setup = False
start_time = time.time()
pip_packages = 'typer rich gmic'
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

if not os.path.isdir('/content/style-transfer-pytorch'):
  !pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
  !git clone https://github.com/olaviinha/style-transfer-pytorch
  !pip install -e ./style-transfer-pytorch

output.clear()
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
op(c.ok, 'Setup finished.')
yeti.timeTaken(start_time)

RENDER=True #@param {type:"boolean"}

if RENDER:
  content_images_dir = "aida/in/trans"
  style_images_dir = "aida/in/style"
  output_dir = "aida/out/trans"
  min_scale = 2112 #@param {type:"slider", min:512, max:2400, step:64}
  max_scale = 2112 #@param {type:"slider", min:512, max:2400, step:64}
  style_scale_factor = 1 #@param {type:"slider", min:0.1, max:10, step:0.1}
  _inputWeight = 0.015 #@param
  _smoothWeight = 2.0 #@param
  _learningRate = 0.03 #@param
  iterations = 500 #@param
  save_every =  100#@param

  # #@markdown <small>Timestamp output files.</small>
  # timestamp = False #@param {type:"boolean"}
  timestamp = False

  remove_iterations = False
  if save_every == 0:
    save_every = 500
    remove_iterations = True

  content_images_dir = fix_path(drive_root+content_images_dir)
  style_images_dir = fix_path(drive_root+style_images_dir)
  output_dir = fix_path(drive_root+output_dir)

  image_min = min_scale
  image_max = max_scale

  import os, random, string, ntpath
  from os import listdir
  from os.path import isfile, join
  from glob import glob
  import IPython
  import requests

  content_imgs = [f for f in listdir(content_images_dir) if isfile(join(content_images_dir, f))]
  style_imgs = [f for f in listdir(style_images_dir) if isfile(join(style_images_dir, f))]

  content_imgs.sort()
  style_imgs.sort()

  if timestamp is True:
    uniq_id = datetime.today().strftime('%Y%m%d%H%M%S')
  else:
    uniq_id = requests.get('https://api.inha.asia/k/?type=short').text

  %cd "{output_dir}"
  for content_img in content_imgs:
    content_img_name = content_img
    content_img = content_images_dir+content_img
    for style_img in style_imgs:
      yeti.txtC(f'Styling {content_img_name}',f'with{style_img}')
      style_img = style_images_dir+style_img
      output_img = output_dir + "TRANS_" + slug(basename(content_img)) + '_' + slug(basename(style_img)) + '__' + uniq_id + '.png'
      !style_transfer "{content_img}" "{style_img}" --output "{output_img}" --save-every {save_every} --min-scale {image_min} --end-scale {image_max} --iterations {iterations} --style-scale-fac {style_scale_factor} -cw {_inputWeight} -tw {_smoothWeight} -ss {_learningRate} --style-weights 0.5
      #!style_transfer "{content_img}" "{style_img}" --output "{output_img}" --save-every {save_every} --min-scale {image_min} --end-scale {image_max} --iterations {iterations} --style-scale-fac {style_scale_factor} --content-weight 0.5 --tv-weight 5.0 --style-weights 0.5 
      IPython.display.Image(output_img)

  if remove_iterations is True:
    output_imgs = [f for f in listdir(output_dir) if isfile(join(output_dir, f))]
    for output_img in output_imgs:
      if '_iter_' in output_img:
        !rm "{output_dir}{output_img}"
        


##### **yeti**Face

In [ ]:
#@markdown > #### **yeti**Face
import os
RENDER=True#@param{type:"boolean"}

if not os.path.isdir('/content/first-order-model'):
  !pip install ffmpy &> /dev/null
  !git init -q .
  !git remote add origin https://github.com/AliaksandrSiarohin/first-order-model
  !git pull -q origin master
  !git clone -q https://github.com/graphemecluster/first-order-model-demo demo
  %pip install imageio-ffmpeg

if RENDER:
	import IPython.display
	import PIL.Image
	import cv2
	import imageio
	import io
	import ipywidgets
	import numpy
	import os.path
	import requests
	import skimage.transform
	import warnings
	from base64 import b64encode
	from demo import load_checkpoints, make_animation
	from ffmpy import FFmpeg
	from google.colab import files, output
	from IPython.display import HTML, Javascript
	from skimage import img_as_ubyte
	warnings.filterwarnings("ignore")
	os.makedirs("user", exist_ok=True)

	display(HTML("""
	<style>
	.widget-box > * {
		flex-shrink: 0;
	}
	.widget-tab {
		min-width: 0;
		flex: 1 1 auto;
	}
	.widget-tab .p-TabBar-tabLabel {
		font-size: 15px;
	}
	.widget-upload {
		background-color: tan;
	}
	.widget-button {
		font-size: 18px;
		width: 160px;
		height: 34px;
		line-height: 34px;
	}
	.widget-dropdown {
		width: 250px;
	}
	.widget-checkbox {
			width: 650px;
	}
	.widget-checkbox + .widget-checkbox {
			margin-top: -6px;
	}
	.input-widget .output_html {
		text-align: center;
		width: 266px;
		height: 266px;
		line-height: 266px;
		color: lightgray;
		font-size: 72px;
	}
	div.stream {
		display: none;
	}
	.title {
		font-size: 20px;
		font-weight: bold;
		margin: 12px 0 6px 0;
	}
	.warning {
		display: none;
		color: red;
		margin-left: 10px;
	}
	.warn {
		display: initial;
	}
	.resource {
		cursor: pointer;
		border: 1px solid gray;
		margin: 5px;
		width: 160px;
		height: 160px;
		min-width: 160px;
		min-height: 160px;
		max-width: 160px;
		max-height: 160px;
		-webkit-box-sizing: initial;
		box-sizing: initial;
	}
	.resource:hover {
		border: 6px solid crimson;
		margin: 0;
	}
	.selected {
		border: 6px solid seagreen;
		margin: 0;
	}
	.input-widget {
		width: 266px;
		height: 266px;
		border: 1px solid gray;
	}
	.input-button {
		width: 268px;
		font-size: 15px;
		margin: 2px 0 0;
	}
	.output-widget {
		width: 256px;
		height: 256px;
		border: 1px solid gray;
	}
	.output-button {
		width: 258px;
		font-size: 15px;
		margin: 2px 0 0;
	}
	.uploaded {
		width: 256px;
		height: 256px;
		border: 6px solid seagreen;
		margin: 0;
	}
	.label-or {
		align-self: center;
		font-size: 20px;
		margin: 16px;
	}
	.loading {
		align-items: center;
		width: fit-content;
	}
	.loader {
		margin: 32px 0 16px 0;
		width: 48px;
		height: 48px;
		min-width: 48px;
		min-height: 48px;
		max-width: 48px;
		max-height: 48px;
		border: 4px solid whitesmoke;
		border-top-color: gray;
		border-radius: 50%;
		animation: spin 1.8s linear infinite;
	}
	.loading-label {
		color: gray;
	}
	.comparison-widget {
		width: 256px;
		height: 256px;
		border: 1px solid gray;
		margin-left: 2px;
	}
	.comparison-label {
		color: gray;
		font-size: 14px;
		text-align: center;
		position: relative;
		bottom: 3px;
	}
	@keyframes spin {
		from { transform: rotate(0deg); }
		to { transform: rotate(360deg); }
	}
	</style>
	"""))

	def thumbnail(file):
		return imageio.get_reader(file, mode='I', format='FFMPEG').get_next_data()

	def create_image(i, j):
		image_widget = ipywidgets.Image(
			value=open('demo/images/%d%d.png' % (i, j), 'rb').read(),
			format='png'
		)
		image_widget.add_class('resource')
		image_widget.add_class('resource-image')
		image_widget.add_class('resource-image%d%d' % (i, j))
		return image_widget

	def create_video(i):
		video_widget = ipywidgets.Image(
			value=cv2.imencode('.png', cv2.cvtColor(thumbnail('demo/videos/%d.mp4' % i), cv2.COLOR_RGB2BGR))[1].tostring(),
			format='png'
		)
		video_widget.add_class('resource')
		video_widget.add_class('resource-video')
		video_widget.add_class('resource-video%d' % i)
		return video_widget

	def create_title(title):
		title_widget = ipywidgets.Label(title)
		title_widget.add_class('title')
		return title_widget

	def download_output(button):
		complete.layout.display = 'none'
		loading.layout.display = ''
		files.download('output.mp4')
		loading.layout.display = 'none'
		complete.layout.display = ''

	def convert_output(button):
		complete.layout.display = 'none'
		loading.layout.display = ''
		FFmpeg(inputs={'output.mp4': None}, outputs={'scaled.mp4': '-vf "scale=1080x1080:flags=lanczos,pad=1920:1080:420:0" -y'}).run()
		files.download('scaled.mp4')
		loading.layout.display = 'none'
		complete.layout.display = ''

	def back_to_main(button):
		complete.layout.display = 'none'
		main.layout.display = ''

	label_or = ipywidgets.Label('or')
	label_or.add_class('label-or')

	image_titles = ['Peoples', 'Cartoons', 'Dolls', 'Game of Thrones', 'Statues']
	image_lengths = [8, 4, 8, 9, 4]

	image_tab = ipywidgets.Tab()
	image_tab.children = [ipywidgets.HBox([create_image(i, j) for j in range(length)]) for i, length in enumerate(image_lengths)]
	for i, title in enumerate(image_titles):
		image_tab.set_title(i, title)

	input_image_widget = ipywidgets.Output()
	input_image_widget.add_class('input-widget')
	upload_input_image_button = ipywidgets.FileUpload(accept='image/*', button_style='primary')
	upload_input_image_button.add_class('input-button')
	image_part = ipywidgets.HBox([
		ipywidgets.VBox([input_image_widget, upload_input_image_button]),
		label_or,
		image_tab
	])

	video_tab = ipywidgets.Tab()
	video_tab.children = [ipywidgets.HBox([create_video(i) for i in range(5)])]
	video_tab.set_title(0, 'All Videos')

	input_video_widget = ipywidgets.Output()
	input_video_widget.add_class('input-widget')
	upload_input_video_button = ipywidgets.FileUpload(accept='video/*', button_style='primary')
	upload_input_video_button.add_class('input-button')
	video_part = ipywidgets.HBox([
		ipywidgets.VBox([input_video_widget, upload_input_video_button]),
		label_or,
		video_tab
	])

	model = ipywidgets.Dropdown(
		description="Model:",
		options=[
			'vox',
			'vox-adv',
			'taichi',
			'taichi-adv',
			'nemo',
			'mgif',
			'fashion',
			'bair'
		]
	)
	warning = ipywidgets.HTML('<b>Warning:</b> Upload your own images and videos (see README)')
	warning.add_class('warning')
	model_part = ipywidgets.HBox([model, warning])

	relative = ipywidgets.Checkbox(description="Relative keypoint displacement (Inherit object proporions from the video)", value=True)
	adapt_movement_scale = ipywidgets.Checkbox(description="Adapt movement scale (Don’t touch unless you know want you are doing)", value=True)
	generate_button = ipywidgets.Button(description="Generate", button_style='primary')
	main = ipywidgets.VBox([
		create_title('Choose Image'),
		image_part,
		create_title('Choose Video'),
		video_part,
		create_title('Settings'),
		model_part,
		relative,
		adapt_movement_scale,
		generate_button
	])

	loader = ipywidgets.Label()
	loader.add_class("loader")
	loading_label = ipywidgets.Label("This may take several minutes to process…")
	loading_label.add_class("loading-label")
	loading = ipywidgets.VBox([loader, loading_label])
	loading.add_class('loading')

	output_widget = ipywidgets.Output()
	output_widget.add_class('output-widget')
	download = ipywidgets.Button(description='Download', button_style='primary')
	download.add_class('output-button')
	download.on_click(download_output)
	convert = ipywidgets.Button(description='Convert to 1920×1080', button_style='primary')
	convert.add_class('output-button')
	convert.on_click(convert_output)
	back = ipywidgets.Button(description='Back', button_style='primary')
	back.add_class('output-button')
	back.on_click(back_to_main)

	comparison_widget = ipywidgets.Output()
	comparison_widget.add_class('comparison-widget')
	comparison_label = ipywidgets.Label('Comparison')
	comparison_label.add_class('comparison-label')
	complete = ipywidgets.HBox([
		ipywidgets.VBox([output_widget, download, convert, back]),
		ipywidgets.VBox([comparison_widget, comparison_label])
	])

	display(ipywidgets.VBox([main, loading, complete]))
	display(Javascript("""
	var images, videos;
	function deselectImages() {
		images.forEach(function(item) {
			item.classList.remove("selected");
		});
	}
	function deselectVideos() {
		videos.forEach(function(item) {
			item.classList.remove("selected");
		});
	}
	function invokePython(func) {
		google.colab.kernel.invokeFunction("notebook." + func, [].slice.call(arguments, 1), {});
	}
	setTimeout(function() {
		(images = [].slice.call(document.getElementsByClassName("resource-image"))).forEach(function(item) {
			item.addEventListener("click", function() {
				deselectImages();
				item.classList.add("selected");
				invokePython("select_image", item.className.match(/resource-image(\d\d)/)[1]);
			});
		});
		images[0].classList.add("selected");
		(videos = [].slice.call(document.getElementsByClassName("resource-video"))).forEach(function(item) {
			item.addEventListener("click", function() {
				deselectVideos();
				item.classList.add("selected");
				invokePython("select_video", item.className.match(/resource-video(\d)/)[1]);
			});
		});
		videos[0].classList.add("selected");
	}, 1000);
	"""))

	selected_image = None
	def select_image(filename):
		global selected_image
		selected_image = resize(PIL.Image.open('demo/images/%s.png' % filename).convert("RGB"))
		input_image_widget.clear_output(wait=True)
		with input_image_widget:
			display(HTML('Image'))
		input_image_widget.remove_class('uploaded')
	output.register_callback("notebook.select_image", select_image)

	selected_video = None
	def select_video(filename):
		global selected_video
		selected_video = 'demo/videos/%s.mp4' % filename
		input_video_widget.clear_output(wait=True)
		with input_video_widget:
			display(HTML('Video'))
		input_video_widget.remove_class('uploaded')
	output.register_callback("notebook.select_video", select_video)

	def resize(image, size=(256, 256)):
			w, h = image.size
			d = min(w, h)
			r = ((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)
			return image.resize(size, resample=PIL.Image.LANCZOS, box=r)

	def upload_image(change):
		global selected_image
		for name, file_info in upload_input_image_button.value.items():
			content = file_info['content']
		if content is not None:
			selected_image = resize(PIL.Image.open(io.BytesIO(content)).convert("RGB"))
			input_image_widget.clear_output(wait=True)
			with input_image_widget:
				display(selected_image)
			input_image_widget.add_class('uploaded')
			display(Javascript('deselectImages()'))
	upload_input_image_button.observe(upload_image, names='value')

	def upload_video(change):
		global selected_video
		for name, file_info in upload_input_video_button.value.items():
			content = file_info['content']
		if content is not None:
			selected_video = 'user/' + name
			preview = resize(PIL.Image.fromarray(thumbnail(content)).convert("RGB"))
			input_video_widget.clear_output(wait=True)
			with input_video_widget:
				display(preview)
			input_video_widget.add_class('uploaded')
			display(Javascript('deselectVideos()'))
			with open(selected_video, 'wb') as video:
				video.write(content)
	upload_input_video_button.observe(upload_video, names='value')

	def change_model(change):
		if model.value.startswith('vox'):
			warning.remove_class('warn')
		else:
			warning.add_class('warn')
	model.observe(change_model, names='value')

	def generate(button):
		main.layout.display = 'none'
		loading.layout.display = ''
		filename = model.value + ('' if model.value == 'fashion' else '-cpk') + '.pth.tar'
		if not os.path.isfile(filename):
			download = requests.get(requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://yadi.sk/d/lEw8uRm140L_eQ&path=/' + filename).json().get('href'))
			with open(filename, 'wb') as checkpoint:
				checkpoint.write(download.content)
		reader = imageio.get_reader(selected_video, mode='I', format='FFMPEG')
		fps = reader.get_meta_data()['fps']
		driving_video = []
		for frame in reader:
			driving_video.append(frame)
		generator, kp_detector = load_checkpoints(config_path='config/%s-256.yaml' % model.value, checkpoint_path=filename)
		predictions = make_animation(
			skimage.transform.resize(numpy.asarray(selected_image), (256, 256)),
			[skimage.transform.resize(frame, (256, 256)) for frame in driving_video],
			generator,
			kp_detector,
			relative=relative.value,
			adapt_movement_scale=adapt_movement_scale.value
		)
		if selected_video.startswith('user/') or selected_video == 'demo/videos/0.mp4':
			imageio.mimsave('temp.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
			FFmpeg(inputs={'temp.mp4': None, selected_video: None}, outputs={'output.mp4': '-c copy -y'}).run()
		else:
			imageio.mimsave('output.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
		loading.layout.display = 'none'
		complete.layout.display = ''
		with output_widget:
			display(HTML('<video id="left" controls src="data:video/mp4;base64,%s" />' % b64encode(open('output.mp4', 'rb').read()).decode()))
		with comparison_widget:
			display(HTML('<video id="right" muted src="data:video/mp4;base64,%s" />' % b64encode(open(selected_video, 'rb').read()).decode()))
		display(Javascript("""
		(function(left, right) {
			left.addEventListener("play", function() {
				right.play();
			});
			left.addEventListener("pause", function() {
				right.pause();
			});
			left.addEventListener("seeking", function() {
				right.currentTime = left.currentTime;
			});
		})(document.getElementById("left"), document.getElementById("right"));
		"""))
		
	generate_button.on_click(generate)

	loading.layout.display = 'none'
	complete.layout.display = 'none'
	select_image('00')
	select_video('0')

#####Utilities

In [ ]:
#-------------------------------------------------------------------------------
#@markdown > ##### **Yeti**Diffuse Prompter
#-------------------------------------------------------------------------------
#YetiDiffuse // PROMPTER
PROMPTER='A $medium of $tics'#@param {type:"string"}
jsonFile='/content/yeti/prompts/prompter_master.json'#@param {type:"string"}
data, text_prompt, strength, prompt_data = yeti.prompter(jsonFile,PROMPTER)
yeti.txtH(text_prompt)

A Marker drawing of a dick straw made of grass in the style of Antipodeans


######**yeti**VideoSplitter

In [ ]:
#@markdown > #### **yeti**VideoSpliter
#-------------------------------------------------------------------------------
videoSplit=False #@param {type:"boolean"}
vid_path = "/content/drive/MyDrive/inter/test.mov" #@param
if videoSplit:
  !mkdir /content/input-frames/
  !ffmpeg -i $vid_path /content/input-frames/frame%09d.png

######**yeti**Copy

In [ ]:
def InOut(Mode,Out,In):
  drivePathIn='/content/MyDrive/aida/in'
  drivePathOut='/content/MyDrive/aida/out'
  DST=f'{drivePathIn}/{yetiIN}'
  SCR=f'{drivePathOut}/{yetiOUT}'
  print(f'{Mode}ing: {SCR}/* \nto: {DST}/*')
  if Mode=='copy': yeti.cp(SCR, DST)
  if Mode=='move': yeti.mv(SCR, DST)
  if Mode=='sync': yeti.sync(SCR, DST)

Mode='sync' #@param ['copy', 'move', 'sync']
yetiOut = 'inter' #@param ['txt2img', 'ken', 'super', 'transfer', 'inter']
yetiIn = 'super' #@param ['init', 'ken', 'super', 'trans', 'inter', 'style']
InOut(Mode,yetiOut,yetiIn)

######yetiTranscode

In [ ]:
#@markdown > #### **yeti**Transcode
#-------------------------------------------------------------------------------
FOLDER='run5'#@param {type:"string"}
videoIn=f'{inPath}/video/{FOLDER}'
videoFiles=glob.glob(f'{videoIn}/*')
if not os.path.isdir('/content/colab-ffmpeg-cuda'):
  !bash /yeti/ini/ffmpegCUDA.sh
for videoFile in videoFiles:
  #-----------------------------------------------------------------------------
  # PARAMETERS / IN / OUT
  start_time = time.time()
  videoIn=videoFile
  videoName=Path(videoIn).stem
  videoPath=f'{outPath}/video/{videoName}'
  videoFile=f'{videoPath}/{videoName}-transcoded_profile.mov'
  yeti.mk(videoPath)
  yeti.txtC('Transcoding', videoIn)
  #-----------------------------------------------------------------------------
  # RENDER / yetiFFMPEG
  !ffmpeg -hwaccel cuda -i $videoIn \
  -hide_banner -loglevel error \
  -c:v prores_ks \
  -profile:v 3 \
  -vendor apch \
  -bits_per_mb 500 \
  -pix_fmt yuv422p10le \
  $videoFile
  #CONSOLE
  SIZE=os.path.getsize(videoFile)
  SIZED=int(SIZE/(1024*1024))
  yeti.txtM('Transcoded', videoFile)
  yeti.txtY('File Size', f'{SIZED} mb')
  yeti.timeTaken(start_time)

####Information

#####**yeti**common
> https://github.com/keirVQGAN/yeti

#####**yeti**Super - information

*   In = /content/drive/MyDrive/aida/in/super
*   Out = /content/drive/MyDrive/aida/out/super
> https://github.com/xinntao/Real-ESRGAN.git  

#####**yeti**Transfer - info

*   In = /content/drive/MyDrive/aida/in/trans
*   Out = /content/drive/MyDrive/aida/out/trans

In [ ]:
#@markdown <---------- **yeti**Transfer // --help
#@markdown > https://github.com/crowsonkb/style-transfer-pytorch
!style_transfer -h

#####**yeti**Ken - info

*   In = /content/drive/MyDrive/aida/in/ken
*   Out = /content/drive/MyDrive/aida/out/ken
*   Config = /content/3d-ken-burns/autozoom.py

> https://github.com/sniklaus/3d-ken-burns.git

#####**yeti**Interpolate - Info

*   In = /content/drive/MyDrive/aida/in/inter
*   Out = /content/drive/MyDrive/aida/out/inter